# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [2]:
# Note that this is not the same db we have used in course videos, please download from this link
# https://drive.google.com/file/d/1O-1-L1DdNxEK6O6nG2jS31MbrMh-OnXM/view?usp=sharing

In [3]:
conn = sqlite3.connect("C:\\Users\\poonam\\Desktop\\FIU\\Applied AI Course\\Assignment 18 - SQL\\Db-IMDB-Assignment.db")
cursor=conn.cursor()

#### Overview of all tables

In [ ]:
### to see all the tables in the database 
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [ ]:
### to see the names of all tables
print(tables)

In [ ]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Updates in Movie table

In [ ]:
### to see the content in the Movie table
cursor.execute("select * from Movie")
result=cursor.fetchall()
#print(result)

In [ ]:
### Update the year column by selecting the last 4 digits and by conveting it into intger values
cursor.execute("update Movie set year=cast(SUBSTR(TRIM(year),-4) as integer)")

In [ ]:
### Remove the trailing spaces,
cursor.execute("update Movie set title=TRIM(title)")

In [ ]:
cursor.execute("update Movie set MID=TRIM(MID)")

In [ ]:
conn.commit()

In [ ]:
conn.close()

## Updates in Genre table 

In [ ]:
### to see the contents in Genre table
cursor.execute("select * from Genre")
result=cursor.fetchall()
#print(result)

In [ ]:
cursor.execute("update Genre set Name=TRIM(Name)")

In [ ]:
conn.commit()

In [ ]:
conn.close()

## updates in Language table 

In [ ]:
cursor.execute("select * from Language")
result=cursor.fetchall()
print(result)

## Updates in Country table 

In [ ]:
cursor.execute("select * from Country")
result=cursor.fetchall()
print(result)

## Updates in Location table

In [ ]:
cursor.execute("select * from Location")
result=cursor.fetchall()
#print(result)

## Updates in M_Cast table

In [ ]:
cursor.execute("select * from M_Cast")
result=cursor.fetchone()
print(result)

In [ ]:
cursor.execute("update M_Cast set MID=TRIM(MID)")

In [ ]:
cursor.execute("update M_Cast set PID=TRIM(PID)")

In [ ]:
conn.commit()

In [ ]:
conn.close()

## Updates in Person table 

In [ ]:
cursor.execute("select * from Person")
result=cursor.fetchall()
print(result)

In [ ]:
cursor.execute("update Person set Gender=TRIM(Gender)")

In [ ]:
cursor.execute("update Person set Name=TRIM(Name)")

In [ ]:
cursor.execute("update Person set PID=TRIM(PID)")

In [ ]:
conn.commit()

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [4]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    assert (q1_results.shape == (232,3))

query1 = """ 

select m.Title,p.Name,m.year
from Movie m 
join M_director d on m.MID = d.MID 
join Person p on d.PID = p.PID 
join M_Genre mg on m.MID = mg.MID 
join Genre g on g.GID = mg.GID
where g.Name like '%Comedy%' AND (m.year%4 = 0 AND m.year % 100 <> 0 OR  m.year % 400 = 0)

"""
grader_1(query1)

                               title              Name  year
0                         Mastizaade      Milap Zaveri  2016
1  Harold & Kumar Go to White Castle      Danny Leiner  2004
2                 Gangs of Wasseypur    Anurag Kashyap  2012
3        Around the World in 80 Days      Frank Coraci  2004
4             The Accidental Husband     Griffin Dunne  2008
5                             Barfi!       Anurag Basu  2012
6                  Bride & Prejudice   Gurinder Chadha  2004
7    Beavis and Butt-Head Do America        Mike Judge  1996
8                            Dostana  Tarun Mansukhani  2008
9                      Kapoor & Sons      Shakun Batra  2016
Wall time: 187 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [5]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))

query2 = """
select p.Name as 'Actor_Names' from Person p 
join 
(select mc.MID,mc.PID from M_cast mc where MID in(select mv.MID from Movie mv where title='Anand' and year=1971)) temp 
on temp.PID=p.PID
"""
grader_2(query2)

        Actor_Names
0     Rajesh Khanna
1  Amitabh Bachchan
2     Sumita Sanyal
3        Ramesh Deo
4         Seema Deo
5    Asit Kumar Sen
6        Dev Kishan
7      Atam Prakash
8     Lalita Kumari
9            Savita
Wall time: 96 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [6]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    return (q3_a.shape == (4942,1)) and (q3_b.shape == (62570,1))

query_less_1970 =""" 
Select p.PID from Person p 
inner join 
(select mc.PID PD, mc.MID from M_cast mc where mc.MID in (select m.MID from Movie m where year<1970)) q1 
on q1.PD=p.PID 
"""
query_more_1990 =""" Select p.PID from Person p 
inner join 
(select mc.PID PD, mc.MID from M_cast mc where mc.MID in (select m.MID from Movie m where year>1990)) q1 
on q1.PD=p.PID 
"""
print(grader_3a(query_less_1970, query_more_1990))

# using the above two queries, you can find the answer to the given question 


(4942, 1)
(62570, 1)
True
Wall time: 387 ms


In [7]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

query3 = """
select p.Name from Person p
join ((select distinct MC.PID from Movie m join M_cast mc on m.MID=mc.MID where year<1970) a
join (select distinct MC.PID from Movie m join M_cast mc on m.MID=mc.MID where year>1990) b on a.PID=b.PID) 
temp1 on p.PID =temp1.PID

"""
grader_3(query3)

               Name
0      Rishi Kapoor
1  Amitabh Bachchan
2            Asrani
3      Zohra Sehgal
4   Parikshat Sahni
5     Rakesh Sharma
6       Sanjay Dutt
7         Ric Young
8             Yusuf
9    Suhasini Mulay
Wall time: 488 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [8]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    return (query_4a.shape == (1462,2))

#*** Write a query, which will return all the directors(id's) along with the number of movies they directed *** 
query_4a =""" 
select md.PID,count(md.PID) 
from M_Director md 
group by md.PID 
Order by count(md.PID) desc
"""
print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

         PID  count(md.PID)
0  nm0223522             39
1  nm0080315             35
2  nm0698184             30
3  nm0890060             30
4  nm0080333             29
5  nm0611531             27
6  nm0007181             21
7  nm0154113             19
8  nm0759662             19
9  nm0007131             18
True
Wall time: 13 ms


In [ ]:
cursor.execute("select md.PID,count(md.PID) from M_Director md group by md.PID Order by count(md.PID) desc")
result=cursor.fetchall()
print(len(result))

In [9]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))

query4 = """ 
select distinct p.Name as Director_Name,n as Movie_Count from Person p
join (select md.PID,count(md.PID) n from M_Director md group by md.PID having n>=10 ) temp
on p.PID= temp.PID
order by n desc
"""
grader_4(query4)

          Director_Name  Movie_Count
0          David Dhawan           39
1          Mahesh Bhatt           35
2       Ram Gopal Varma           30
3          Priyadarshan           30
4          Vikram Bhatt           29
5  Hrishikesh Mukherjee           27
6           Yash Chopra           21
7        Shakti Samanta           19
8       Basu Chatterjee           19
9          Subhash Ghai           18
Wall time: 30 ms


In [ ]:
cursor.execute("select distinct p.Name from Person p join (select md.PID,count(md.PID) n from M_Director md group by md.PID having n>=10 ) temp on p.PID= temp.PID order by n desc")
result=cursor.fetchall()
print(len(result))

## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [ ]:
cursor.execute("select p.Name,p.Gender from Person p join (select mc.MID,mc.PID from M_cast mc where MID in(select mv.MID from Movie mv)) temp on temp.PID=p.PID")
result=cursor.fetchall()
print(result)

In [ ]:
#cursor.execute("select year, count(MID) from Movie group by year")
cursor.execute("select mc.MID,p.Gender from Person p join M_cast mc on p.PID=mc.PID where Gender='Female'")
result=cursor.fetchone()
print(result)
#print(len(result))

In [ ]:
#cursor.execute("select year, count(MID) from Movie group by year")
cursor.execute("select m.year,count(m.title),temp.Gender from Movie m join (select mc.MID,p.Gender from Person p join M_cast mc on p.PID=mc.PID where Gender='Female') temp on m.MID=temp.MID group by m.year")
result=cursor.fetchall()
print(result)
#print(len(result))

In [10]:
%%time

# note that you don't need TRIM for person table

# *** Write your query that will get movie id, and number of people for each geneder ***

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10)) 
    return (query_5aa.shape == (8846,3))

query_5aa =""" 

select mc.MID, p.Gender, count(Gender) from M_cast mc
join Person p
on mc.PID=p.PID
group by mc.MID,p.Gender

"""
print(grader_5aa(query_5aa))

# *** Write your query that will have at least one male actor try to use query that you have written above ***
def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10)) 
    return (query_5ab.shape == (3469, 3))

query_5ab =""" 
select mc.MID, p.Gender, count(Gender) from M_cast mc
join Person p
on mc.PID=p.PID
group by mc.MID,p.Gender
having p.Gender='Male'
"""

print(grader_5ab(query_5ab))

# using the above queries, you can write the answer to the given question

         MID  Gender  count(Gender)
0  tt0021594    None              0
1  tt0021594  Female              3
2  tt0021594    Male              5
3  tt0026274    None              0
4  tt0026274  Female             11
5  tt0026274    Male              9
6  tt0027256    None              0
7  tt0027256  Female              5
8  tt0027256    Male              8
9  tt0028217  Female              3
True
         MID Gender  count(Gender)
0  tt0021594   Male              5
1  tt0026274   Male              9
2  tt0027256   Male              8
3  tt0028217   Male              7
4  tt0031580   Male             27
5  tt0033616   Male             46
6  tt0036077   Male             11
7  tt0038491   Male              7
8  tt0039654   Male              6
9  tt0040067   Male             10
True
Wall time: 581 ms


In [11]:
%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    assert (q5a_results.shape == (4,2))

query5a = """ 
select m.year,count(m.title) one from Movie m
join
(select mc.MID, p.Gender, count(Gender) as cnt from M_cast mc
join Person p on mc.PID=p.PID group by mc.MID,p.Gender) temp
on m.MID=temp.MID
group by m.title 
having temp.Gender='Female' and one=1

"""
grader_5a(query5a)

   year  one
0  1999    1
1  1939    1
2  2018    1
3  2000    1
Wall time: 288 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [12]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(100))
    assert (q5b_results.shape == (4,3))

query5b = """ 

select q1.year,q2.total,(q1.Female_Cast_Only_Movies*1.00/q2.total)  
from 
(   
    select m.year,count(m.title) Female_Cast_Only_Movies from Movie m
    join
    (select mc.MID, p.Gender, count(Gender) as cnt from M_cast mc
    join Person p on mc.PID=p.PID group by mc.MID,p.Gender) temp
    on m.MID=temp.MID
    group by m.title 
    having temp.Gender like 'Female' and Female_Cast_Only_Movies =1
)q1

join
(
    select year, count(MID) total
    from Movie
    group by year
)q2
on q1.year=q2.year

"""
grader_5b(query5b)


   year  total  (q1.Female_Cast_Only_Movies*1.00/q2.total)
0  1999     66                                    0.015152
1  1939      2                                    0.500000
2  2018    104                                    0.009615
3  2000     64                                    0.015625
Wall time: 334 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [13]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))

query6 = """ 
select m.title,count(temp.PID)  from Movie m 
join  (select  distinct mc.PID,m.MID  from Movie m 
       join M_Cast mc on m.MID=mc.MID group by m.MID,mc.PID) temp 
on m.MID=temp.MID
group by m.MID
order by count(temp.PID) desc 
"""
grader_6(query6)
#join  M_Cast mc on m.MID=mc.MID

                        title  count(temp.PID)
0               Ocean's Eight              238
1                    Apaharan              233
2                        Gold              215
3             My Name Is Khan              213
4  Captain America: Civil War              191
5                    Geostorm              170
6                     Striker              165
7                        2012              154
8                      Pixels              144
9       Yamla Pagla Deewana 2              140
Wall time: 728 ms


In [ ]:
#cursor.execute("select distinct m.title,mc.MID  from Movie m join M_Cast mc on m.MID=mc.MID  ")
cursor.execute("select m.title,count(temp.PID)  from Movie m join  (select  distinct mc.PID,m.MID  from Movie m join M_Cast mc on m.MID=mc.MID group by mc.PID,m.MID) temp on m.MID=temp.MID group by m.title order by count(temp.PID) desc")
result=cursor.fetchall()
#print(result)
#(select distinct mc.PID,mc.MID  from Movie m join M_Cast mc on m.MID=mc.MID)

In [ ]:
print(len(result))

### Q7 --- A decade is a sequence of 10 consecutive years. 
### For example, say in your database you have movie information starting from 1931. 
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on. 
### Find the decade D with the largest number of films and the total number of films in D

In [14]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))

query7a = """

select count(MID), year from Movie 
group by year

"""
grader_7a(query7a)

#*** Write a query that computes number of movies in each year ***
# using the above query, you can write the answer to the given question

   count(MID)  year
0           1  1931
1           3  1936
2           2  1939
3           1  1941
4           1  1943
5           2  1946
6           2  1947
7           3  1948
8           3  1949
9           2  1950
Wall time: 8.99 ms


In [15]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results.head(10))
    assert (q7b_results.shape == (713, 4))

query7b = """   

select t1.year,t1.cnt,t2.year,t2.cnt
from
    (select year, count(title) as cnt from Movie 
    group by year) t1
join 
    (select year, count(title) as cnt from Movie 
    group by year) t2
on t2.year<=t1.year+9 and (t2.year>t1.year or t2.year=t1.year)
        
"""
grader_7b(query7b)
# if you see the below results the first movie year is less than 2nd movie year and 
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question
# Write a query that will do joining of the above table(7a) with itself 
# such that you will join with only rows if the second tables year is <= current_year+9 and more than or equal current_year

   year  cnt  year  cnt
0  1931    1  1931    1
1  1931    1  1936    3
2  1931    1  1939    2
3  1936    3  1936    3
4  1936    3  1939    2
5  1936    3  1941    1
6  1936    3  1943    1
7  1939    2  1939    2
8  1939    2  1941    1
9  1939    2  1943    1
Wall time: 21 ms


In [16]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

query7 = """ 

with
    decade_cal as
    (select distinct year, year  decade_start, year+9  decade_end,year decade
    from Movie),

    movies_in_year as  
    (select cast(substr(year,-4) as unsigned) year, count(MID) no_of_movies 
    from Movie 
    group by cast(substr(year,-4) as unsigned)), 
    
    movies_in_decade as
    (select sum(no_of_movies) total ,decade_cal.decade
    from decade_cal,movies_in_year
    where movies_in_year.year between decade_cal.decade_start and decade_cal.decade_end
    group by decade_cal.decade)
    
    select decade,total from  movies_in_decade
    where total=(select max(total) from  movies_in_decade)
       
"""
grader_7(query7)
# if you check the output we are printinng all the year in that decade, its fine you can print 2008 or 2008-2017

  decade  total
0   2008   1203
Wall time: 26 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [17]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

query8a = """

select mc.PID as Actor,md.PID as Director,count(md.MID) as movies
from M_Director md join M_Cast mc on md.MID=mc.MID
group by md.PID,mc.PID

"""
grader_8a(query8a)

# using the above query, you can write the answer to the given question
#*** Write a query that will results in number of movies actor-director worked together ***

       Actor   Director  movies
0  nm0000027  nm0000180       1
1  nm0001114  nm0000180       1
2  nm0001919  nm0000180       1
3  nm0006762  nm0000180       1
4  nm0030062  nm0000180       1
5  nm0038970  nm0000180       1
6  nm0051856  nm0000180       1
7  nm0085966  nm0000180       1
8  nm0097889  nm0000180       1
9  nm0125497  nm0000180       1
Wall time: 488 ms


In [ ]:
cursor.execute("select * from Person where Name ='Yash Chopra'")
result=cursor.fetchall()
print(result)

In [ ]:
cursor.execute("select * from Person where Name ='Ravikant'")
result=cursor.fetchall()
print(result)

In [18]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

query8 = """

select p.Name,q1.m
from Person p
join
(select a,m 
from(select mc.PID a ,md.PID d,count(md.MID) as m
    from M_Director md 
    join M_Cast mc on md.MID=mc.MID
    group by md.PID,mc.PID
    ) 
where (a,m)
in 
(select a,max(m)
    from 
        (
        select mc.PID a ,md.PID d,count(md.MID) as m
        from M_Director md 
        join M_Cast mc on md.MID=mc.MID
        group by md.PID,mc.PID
        )     
group by a)
and d='nm0007181'
order by m desc) q1
on p.PID=q1.a


"""
grader_8(query8)

               Name   m
0       Jagdish Raj  11
1  Manmohan Krishna  10
2          Iftekhar   9
3     Shashi Kapoor   7
4    Waheeda Rehman   5
5     Rakhee Gulzar   5
6    Achala Sachdev   4
7       Neetu Singh   4
8          Ravikant   4
9   Parikshat Sahni   3
(245, 2)
Wall time: 711 ms


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [ ]:
cursor.execute("select * from Person where Name like '%shah Rukh Khan%'")
result=cursor.fetchall()
print(result)

In [ ]:
cursor.execute("select distinct MID from M_Cast mc join Person p on mc.PID=p.PID where p.Name like '%shah Rukh Khan%'")
result=cursor.fetchall()
print(len(result))

In [19]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

query9a = """ 
   
select distinct PID from M_cast 
join
(select distinct MID from M_Cast mc join Person p on mc.PID=p.PID where p.Name like '%shah Rukh Khan%'
) q1
on M_cast.MID=q1.MID
where PID not like 'nm0451321'

"""
grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors
####comment by Poonam -- exclude shah Rukh Khan from list

         PID
0  nm0004418
1  nm1995953
2  nm2778261
3  nm0631373
4  nm0241935
5  nm0792116
6  nm1300111
7  nm0196375
8  nm1464837
9  nm2868019
(2382, 1)
Wall time: 264 ms


In [20]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

query9 = """ 

select name from Person
join
(
    select PID from
    (
    select distinct M_Cast.PID from M_Cast
    join
    (select pid,mid from m_cast 
        where pid in
            (select pid from m_cast where mid in
            (select mid from m_cast where pid in
            (select pid from person where name like "%shah Rukh Khan%")))
        )q1
        on M_cast.MID=q1.MID
    ) 

    where PID not in
    (
    select M_Cast.PID from M_Cast
    join
        (select * from Person p
        join M_Cast mc on p.PID=mc.PID 
        join Movie m on mc.MID=m.MID
        where p.Name like '%Shah Rukh Khan%') q1
        on M_cast.MID=q1.MID
        )
) temp
on Person.PID=temp.PID

"""
grader_9(query9)

                    Name
0           Freida Pinto
1            Rohan Chand
2           Damian Young
3        Waris Ahluwalia
4  Caroline Christl Long
5          Rajeev Pahuja
6      Michelle Santiago
7        Alicia Vikander
8           Dominic West
9         Walton Goggins
(25698, 1)
Wall time: 1.63 s
